In [12]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [13]:
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [14]:
%pip install playwright

Note: you may need to restart the kernel to use updated packages.


In [15]:
!playwright install 

In [92]:
SEASONS=list(range(2022,2025))
SEASONS

[2022, 2023, 2024]

In [93]:
DATA_DIR= 'data2'
STANDINGS_DIR= os.path.join(DATA_DIR, 'standings')
SCORES_DIR= os.path.join(DATA_DIR, 'scores')

In [94]:
async def get_html(url, selector, sleep=5, retries=3):
    html=None
    for i in range(1, retries+1):
        time.sleep(sleep * i)
        
        try:
            async with async_playwright() as p:
                browser= await p.chromium.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title)
                html= await page.inner_html(selector)
        except playwrightTimeout:
            print(f"Timout error on {url}")
            continue 
        else:
            break
    return html

In [95]:
async def get_html(url, selector, sleep=5, retries=5):
    html = None
    for i in range(1, retries + 1):
        time.sleep(sleep * i)
        print(f"Fetching {url}, Attempt {i}")
        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch()
                page = await browser.new_page()
                await page.goto(url, timeout=60000)  # Increase the timeout to 60,000 ms (60 seconds)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:  # Correct the exception name
            print(f"Timeout error on {url}")
        else:
            break
    return html


In [96]:
async def scrape_season(season):
    url= f'https://www.basketball-reference.com/leagues/NBA_{season}_games.html'
    html= await get_html(url, '#content .filter')
    
    soup=BeautifulSoup(html)
    links=soup.find_all('a')
    href=[l['href']for l in links]
    standings_pages= [f'https://www.basketball-reference.com{l}'for l in href]
    
    for url in standings_pages:
        save_path= os.path.join(STANDINGS_DIR, url.split('/')[-1])
        if os.path.exists(save_path):
            continue
            
        html = await get_html(url,'#all_schedule')
        with open (save_path, 'w+') as f:
            f.write(html)

In [64]:
for season in SEASONS:
    await scrape_season(season)

Fetching https://www.basketball-reference.com/leagues/NBA_2022_games.html, Attempt 1
2021-22 NBA Schedule | Basketball-Reference.com
Fetching https://www.basketball-reference.com/leagues/NBA_2022_games-october.html, Attempt 1
2021-22 NBA Schedule | Basketball-Reference.com
Fetching https://www.basketball-reference.com/leagues/NBA_2022_games-november.html, Attempt 1
2021-22 NBA Schedule | Basketball-Reference.com
Fetching https://www.basketball-reference.com/leagues/NBA_2022_games-december.html, Attempt 1
2021-22 NBA Schedule | Basketball-Reference.com
Fetching https://www.basketball-reference.com/leagues/NBA_2022_games-january.html, Attempt 1
2021-22 NBA Schedule | Basketball-Reference.com
Fetching https://www.basketball-reference.com/leagues/NBA_2022_games-february.html, Attempt 1
Timeout error on https://www.basketball-reference.com/leagues/NBA_2022_games-february.html
Fetching https://www.basketball-reference.com/leagues/NBA_2022_games-february.html, Attempt 2
2021-22 NBA Schedule |

In [97]:
standing_files= os.listdir(STANDINGS_DIR)

In [98]:
standing_files

['NBA_2022_games-october.html',
 'NBA_2023_games-may.html',
 'NBA_2023_games-april.html',
 'NBA_2022_games-may.html',
 'NBA_2023_games-january.html',
 'NBA_2024_games-march.html',
 'NBA_2024_games-november.html',
 'NBA_2024_games-december.html',
 'NBA_2022_games-april.html',
 'NBA_2022_games-december.html',
 'NBA_2023_games-november.html',
 'NBA_2024_games-october.html',
 'NBA_2023_games-december.html',
 'NBA_2022_games-november.html',
 'NBA_2022_games-march.html',
 'NBA_2023_games-february.html',
 'NBA_2022_games-february.html',
 'NBA_2024_games-february.html',
 'NBA_2022_games-june.html',
 'NBA_2024_games-january.html',
 'NBA_2022_games-january.html',
 'NBA_2024_games-april.html',
 'NBA_2023_games-june.html',
 'NBA_2023_games-march.html',
 'NBA_2023_games-october.html']

In [99]:
async def scrape_game(standing_files):
    with open(standing_files, 'r') as f:
        html= f.read()

    soup=BeautifulSoup(html)
    links= soup.find_all('a')
    hrefs=[l.get('href')for l in links]
    box_scores= [l for l in hrefs if l and "boxscore" in l and '.html' in l]
    box_scores= [f"https://www.basketball-reference.com{l}" for l in box_scores]

    for url in box_scores:
        save_path= os.path.join(SCORES_DIR, url.split('/')[-1])
        if os.path.exists(save_path):
            continue

        html= await get_html(url, "#content")
        if not html:
            continue
        with open(save_path, 'w+')as f:
            f.write(html)

In [100]:
#chat gpt
async def scrape_game(standing_files):
    with open(standing_files, 'r') as f:
        html= f.read()

    soup=BeautifulSoup(html)
    links= soup.find_all('a')
    hrefs=[l.get('href')for l in links]
    box_scores= [l for l in hrefs if l and "boxscore" in l and '.html' in l]
    box_scores= [f"https://www.basketball-reference.com{l}" for l in box_scores]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split('/')[-1])
        
        # Create the directory if it doesn't exist
        os.makedirs(os.path.dirname(save_path), exist_ok=True)

        if os.path.exists(save_path):
            continue

        html = await get_html(url, '#content')
        if not html:
            continue 

        with open(save_path, 'w+') as f:
            f.write(html)

In [89]:
# Your existing standing_files list
standing_files = ['NBA_2022_games-october.html', 'NBA_2023_games-may.html', 'NBA_2023_games-april.html', 'NBA_2022_games-may.html', 'NBA_2023_games-january.html', 'NBA_2024_games-march.html', 'NBA_2024_games-november.html', 'NBA_2024_games-december.html', 'NBA_2022_games-april.html', 'NBA_2022_games-december.html', 'NBA_2023_games-november.html', 'NBA_2024_games-october.html', 'NBA_2023_games-december.html', 'NBA_2022_games-november.html', 'NBA_2022_games-march.html', 'NBA_2023_games-february.html', 'NBA_2022_games-february.html', 'NBA_2024_games-february.html', 'NBA_2022_games-june.html', 'NBA_2024_games-january.html', '.ipynb_checkpoints', 'NBA_2022_games-january.html', 'NBA_2024_games-april.html', 'NBA_2023_games-june.html', 'NBA_2023_games-march.html', 'NBA_2023_games-october.html']

# Remove '.ipynb_checkpoints' from the list
standing_files = [file for file in standing_files if file != '.ipynb_checkpoints']

# Now 'standing_files' contains the updated list without '.ipynb_checkpoints'
print(standing_files)


['NBA_2022_games-october.html', 'NBA_2023_games-may.html', 'NBA_2023_games-april.html', 'NBA_2022_games-may.html', 'NBA_2023_games-january.html', 'NBA_2024_games-march.html', 'NBA_2024_games-november.html', 'NBA_2024_games-december.html', 'NBA_2022_games-april.html', 'NBA_2022_games-december.html', 'NBA_2023_games-november.html', 'NBA_2024_games-october.html', 'NBA_2023_games-december.html', 'NBA_2022_games-november.html', 'NBA_2022_games-march.html', 'NBA_2023_games-february.html', 'NBA_2022_games-february.html', 'NBA_2024_games-february.html', 'NBA_2022_games-june.html', 'NBA_2024_games-january.html', 'NBA_2022_games-january.html', 'NBA_2024_games-april.html', 'NBA_2023_games-june.html', 'NBA_2023_games-march.html', 'NBA_2023_games-october.html']


In [102]:
for f in standing_files:
    filepath= os.path.join(STANDINGS_DIR, f)
    
    await scrape_game(filepath)

Fetching https://www.basketball-reference.com/boxscores/202201120SAS.html, Attempt 1
Rockets vs Spurs, January 12, 2022 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202201120UTA.html, Attempt 1
Cavaliers vs Jazz, January 12, 2022 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202201120CHI.html, Attempt 1
Nets vs Bulls, January 12, 2022 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202201120SAC.html, Attempt 1
Lakers vs Kings, January 12, 2022 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202201130MIL.html, Attempt 1
Warriors vs Bucks, January 13, 2022 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202201130MEM.html, Attempt 1
Timberwolves vs Grizzlies, January 13, 2022 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202201130NOP.html, Attempt 1
Timeout error on https://w

Fetching https://www.basketball-reference.com/boxscores/202303050DAL.html, Attempt 1
Suns vs Mavericks, March 5, 2023 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202303050CHI.html, Attempt 1
Pacers vs Bulls, March 5, 2023 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202303050LAL.html, Attempt 1
Warriors vs Lakers, March 5, 2023 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202303050BRK.html, Attempt 1
Hornets vs Nets, March 5, 2023 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202303050ORL.html, Attempt 1
Trail Blazers vs Magic, March 5, 2023 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202303050HOU.html, Attempt 1
Spurs vs Rockets, March 5, 2023 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202303050OKC.html, Attempt 1
Jazz vs Thunder, March 5, 2023 | Basketball-

Bucks vs Warriors, March 11, 2023 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202303110NOP.html, Attempt 1
Thunder vs Pelicans, March 11, 2023 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202303110PHO.html, Attempt 1
Kings vs Suns, March 11, 2023 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202303120DEN.html, Attempt 1
Nets vs Nuggets, March 12, 2023 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202303120CHO.html, Attempt 1
Cavaliers vs Hornets, March 12, 2023 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202303120PHI.html, Attempt 1
Wizards vs 76ers, March 12, 2023 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202303120NOP.html, Attempt 1
Timeout error on https://www.basketball-reference.com/boxscores/202303120NOP.html
Fetching https://www.basketball-reference.co

Magic vs Clippers, March 18, 2023 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202303180IND.html, Attempt 1
76ers vs Pacers, March 18, 2023 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202303180TOR.html, Attempt 1
Timeout error on https://www.basketball-reference.com/boxscores/202303180TOR.html
Fetching https://www.basketball-reference.com/boxscores/202303180TOR.html, Attempt 2
Timberwolves vs Raptors, March 18, 2023 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202303180CHI.html, Attempt 1
Heat vs Bulls, March 18, 2023 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202303180MEM.html, Attempt 1
Warriors vs Grizzlies, March 18, 2023 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202303180WAS.html, Attempt 1
Kings vs Wizards, March 18, 2023 | Basketball-Reference.com
Fetching https://www.basketball-referen

Fetching https://www.basketball-reference.com/boxscores/202303240SAC.html, Attempt 3
Suns vs Kings, March 24, 2023 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202303240LAL.html, Attempt 1
Thunder vs Lakers, March 24, 2023 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202303250ATL.html, Attempt 1
Pacers vs Hawks, March 25, 2023 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202303250MIA.html, Attempt 1
Nets vs Heat, March 25, 2023 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202303250DEN.html, Attempt 1
Bucks vs Nuggets, March 25, 2023 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202303250PHO.html, Attempt 1
76ers vs Suns, March 25, 2023 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202303250SAC.html, Attempt 1
Jazz vs Kings, March 25, 2023 | Basketball-Reference.co

Celtics vs Bucks, March 30, 2023 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202303300DEN.html, Attempt 1
Pelicans vs Nuggets, March 30, 2023 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202303310CHO.html, Attempt 1
Bulls vs Hornets, March 31, 2023 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202303310IND.html, Attempt 1
Thunder vs Pacers, March 31, 2023 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202303310PHI.html, Attempt 1
Raptors vs 76ers, March 31, 2023 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202303310WAS.html, Attempt 1
Magic vs Wizards, March 31, 2023 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202303310BOS.html, Attempt 1
Jazz vs Celtics, March 31, 2023 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202303310BR

Rockets vs Bucks, October 22, 2022 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202210220DAL.html, Attempt 1
Grizzlies vs Mavericks, October 22, 2022 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202210220DEN.html, Attempt 1
Thunder vs Nuggets, October 22, 2022 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202210220SAC.html, Attempt 1
Clippers vs Kings, October 22, 2022 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202210230LAL.html, Attempt 1
Trail Blazers vs Lakers, October 23, 2022 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202210230ATL.html, Attempt 1
Hornets vs Hawks, October 23, 2022 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202210230CLE.html, Attempt 1
Wizards vs Cavaliers, October 23, 2022 | Basketball-Reference.com
Fetching https://www.basketball-refe

Fetching https://www.basketball-reference.com/boxscores/202210290CHO.html, Attempt 1
Warriors vs Hornets, October 29, 2022 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202210290BRK.html, Attempt 1
Pacers vs Nets, October 29, 2022 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202210290CHI.html, Attempt 1
76ers vs Bulls, October 29, 2022 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202210290MIL.html, Attempt 1
Hawks vs Bucks, October 29, 2022 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202210290DAL.html, Attempt 1
Thunder vs Mavericks, October 29, 2022 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202210290UTA.html, Attempt 1
Grizzlies vs Jazz, October 29, 2022 | Basketball-Reference.com
Fetching https://www.basketball-reference.com/boxscores/202210300LAC.html, Attempt 1
Pelicans vs Clippers, October 3

In [90]:
standing_files

['NBA_2022_games-october.html',
 'NBA_2023_games-may.html',
 'NBA_2023_games-april.html',
 'NBA_2022_games-may.html',
 'NBA_2023_games-january.html',
 'NBA_2024_games-march.html',
 'NBA_2024_games-november.html',
 'NBA_2024_games-december.html',
 'NBA_2022_games-april.html',
 'NBA_2022_games-december.html',
 'NBA_2023_games-november.html',
 'NBA_2024_games-october.html',
 'NBA_2023_games-december.html',
 'NBA_2022_games-november.html',
 'NBA_2022_games-march.html',
 'NBA_2023_games-february.html',
 'NBA_2022_games-february.html',
 'NBA_2024_games-february.html',
 'NBA_2022_games-june.html',
 'NBA_2024_games-january.html',
 'NBA_2022_games-january.html',
 'NBA_2024_games-april.html',
 'NBA_2023_games-june.html',
 'NBA_2023_games-march.html',
 'NBA_2023_games-october.html']